In [9]:
# CS124 Programming Assignment 3
# Christian Hallas
# Spring 2016

In [10]:
# Import packages
import heapq
import random
import math
import time

In [11]:
# Constants defined
NO_INTS = 100
MIN_INT = 1
MAX_INT = 10 ** 12
IND_LST = range(NO_INTS)

In [12]:
# Generate a list of length 'size' of integers in range [0,10^12]
def random_ints(size, lower=MIN_INT, upper=MAX_INT):
    return [random.randrange(lower, upper+1) for i in range(size)]

In [13]:
# Implementation of the Karmarkar-Karp algorithm using a heap
def karmarkarkarp(A=[]):
    if not A:
        A = random_ints(NO_INTS)
    A_neg = [-x for x in A]
    heapq.heapify(A_neg)
    for _ in range(NO_INTS-1):
        max_int1 = heapq.heappop(A_neg)
        max_int2 = heapq.heappop(A_neg)
        heapq.heappush(A_neg, max_int1-max_int2)
    return -A_neg[0]

In [14]:
# Generate random standard representation solution
def rand_std():
    rand_lst = random_ints(NO_INTS, lower=0, upper=1)
    return [x if x == 1 else -1 for x in rand_lst]

In [15]:
# Generate random prepartition representation solution
def rand_pre():
    return random_ints(NO_INTS, lower=0, upper=NO_INTS-1)

In [16]:
# Generate A' representation from prepartition representation P and list of integers A
def A_prime_repr(A, P):
    A_prime = [0] * NO_INTS
    for i in range(NO_INTS):
        A_prime[P[i]] += A_prime[P[i]] + A[i]
    return A_prime

In [17]:
# Residue from standard representation solution P
def res_std(A, P):
    return abs(sum([x*y for x,y in zip(A,P)]))

In [18]:
# Residue from prepartition representation solution P
def res_pre(A, P):
    return karmarkarkarp(A=A_prime_repr(A,P))

In [19]:
# Move to random neighbor from standard solution P
def move_std(P):
    P_p = P[:]
    inds = random.sample(IND_LST, 2)
    P_p[inds[0]] *= -1
    if random.randint(0,1):
        P_p[inds[1]] *= -1
    return P_p

In [20]:
# Move to random neighbor from prepartition solution P
def move_pre(P):
    P_p = P[:]
    inds = [0,0]
    while inds[0] == inds[1]:
        inds = random.sample(IND_LST, 2)
    P_p[inds[0]] = inds[1]
    return P_p

In [21]:
# Implementation of the repeated random algorithm
def repeat_rand(iterations, reps, A=[]):
    if not A:
        A = random_ints(NO_INTS)
    if reps == "std":
        res = res_std
        rand = rand_std
    if reps == "pre":
        res = res_pre
        rand = rand_pre
    P = rand()
    S = res(A, P)
    for _ in range(iterations):
        P = rand()
        S_prime = res(A, P)
        if S > S_prime:
            S = S_prime
    return S

In [22]:
# Implementation of the hill climbing algorithm
def hill_climb(iterations, reps, A=[]):
    if not A:
        A = random_ints(NO_INTS)
    if reps == "std":
        res = res_std
        rand = rand_std
        move = move_std
    if reps == "pre":
        res = res_pre
        rand = rand_pre
        move = move_pre
    P = rand()
    S = res(A, P)
    for _ in range(iterations):
        P_p = move(P)
        S_p = res(A, P_p)
        if S > S_p:
            S = S_p
            P = P_p
    return S

In [23]:
# Define cooling schdule
def T(i):
    return (10 ** 10) * (0.8 ** math.floor(float(i)/300))

In [24]:
# Implementation of the simulated annealing
def sim_ann(iterations, reps, A=[]):
    if not A:
        A = random_ints(NO_INTS)
    if reps == "std":
        res = res_std
        rand = rand_std
        move = move_std
    if reps == "pre":
        res = res_pre
        rand = rand_pre
        move = move_pre
    P = rand()
    S = res(A, P)
    S_pp = S
    P_pp = P
    for i in range(iterations):
        P_p = move(P)
        S_p = res(A, P_p)
        if S > S_p:
            S = S_p
            P = P_p
        else:
            prob = math.exp(-float(res(A, P_p) - res(A, P)) / T(i+1))
            if random.uniform(0,1) <= prob:
                S = S_p
                P = P_p
        if S_pp > S:
            S_pp = S
            P_pp = P
    return S_pp

In [25]:
print("Repeat std:", repeat_rand(25000, "std"))
print("Repeat pre:", repeat_rand(25000, "pre"))
print("Hill std:", hill_climb(25000, "std"))
print("Hill pre:", hill_climb(25000, "pre"))
print("Sim std:", sim_ann(25000, "std"))
print("Sim pre:", sim_ann(25000, "pre"))

Repeat std: 317577203
Repeat pre: 137
Hill std: 1684737909
Hill pre: 2138
Sim std: 462660999
Sim pre: 199
